# python_scraper

In [12]:
import re
import sqlite3
from urllib.request import urlopen
from html import unescape

In [13]:
def main():
    """
    메인 처리
    fetch(), scrape(), save() 함수를 호출
    """
    html = fetch('http://www.hanbit.co.kr/store/books/full_book_list.html')
    books = scrape(html)
    save('books.db', books)

In [14]:
def fetch(url):
    """
    매개변수로 전달받을 url을 기반으로 웹 페이지를 추출
    웹 페이지의 Content-Type 헤더를 통해 인코딩 형식 확인
    반환값: str 자료형의 HTML
    """
    f = urlopen(url)
    # HTTP 헤더를 기반으로 인코딩 형식 추출
    encoding = f.info().get_content_charset(failobj="utf-8")
    # 추출한 인코딩 형식을 기반으로 문자열 디코딩
    html = f.read().decode(encoding)
    return html

In [15]:
def scrape(html):
    """
    매개변수 html로 받은 HTML을 기반으로 정규 표현식을 사용해 도서 정보를 추출합니다.
    반환값: 도서(dict) 리스트
    """
    books = []
    # re.findall()을 사용해 도서 하나에 해당하는 HTML을 추출
    for partial_html in re.findall(r'<td class="left"><a.*?</td>', html, re.DOTALL):
        # 도서의 URL을 추출
        url = re.search(r'<a href="(.*?)">', partial_html).group(1)
        url = 'http://www.hanbit.co.kr' + url
        # 태그를 제거해서 도서의 제목 추출
        title = re.sub(r'<.*?>', '', partial_html)
        title = unescape(title)
        books.append({'url': url, 'title': title})
    
    return books

In [28]:
def save(db_path, books):
    """
    매개변수 books로 전달된 도서 목록을 SQLite 데이터베이스에 저장
    데이터베이스의 경로는 매개변수 dp_path로 지정
    반환값: None(없음)
    """
    # DB Connection
    conn = sqlite3.connect(db_path)
    # 커서 추출
    c = conn.cursor()
    # execute() 메서드로 SQL을 실행
    # 스크립트를 여러 번 실행할 수 있으므로 기존의 books 테이블을 제거
    c.execute('DROP TABLE IF EXISTS books')
    # books 테이블을 생성합니다.
    c.execute('''
        CREATE TABLE books (
            title text,
            url text
        )
    ''')
    # executemany() 메서드를 사용하면 매개변수로 리스트를 지정
    c.executemany('INSERT INTO books VALUES (:title, :url)', books)
    # 변경사항을 커밋
    conn.commit()
    # DB Connection 종료
    conn.close()

In [29]:
# python 명령어로 실행한 경우 main() 함수를 호출
# 모듈로써 다른 파일에서 읽어 들였을 때 main() 함수가 호출되지 않도록 함
# 파이썬 프로그램의 일반적인 작성 방식
if __name__ == '__main__':
    main()

In [30]:
# 저장된 내용 확인
with sqlite3.connect('books.db') as conn:
    c = conn.cursor()
    c.execute('SELECT * FROM books')
    for row in c.fetchall():
        print(row)

('받침 없는 한글 동화 : 무시무시 마녀가 이사 와!', 'http://www.hanbit.co.kr/store/books/look.php?p_code=B8661740335')
('받침 없는 한글 동화 : 도, 도, 도깨비다!', 'http://www.hanbit.co.kr/store/books/look.php?p_code=B6084607806')
('받침 없는 한글 동화 : 바쁘다 바빠 너구리 바빠', 'http://www.hanbit.co.kr/store/books/look.php?p_code=B1382037871')
('받침 없는 한글 동화 : 도깨비 파자마 파티', 'http://www.hanbit.co.kr/store/books/look.php?p_code=B8948111854')
('받침 없는 한글 동화 : 4권 세트', 'http://www.hanbit.co.kr/store/books/look.php?p_code=B3178834938')
('오늘도 시작하지 못하는 당신을 위해', 'http://www.hanbit.co.kr/store/books/look.php?p_code=B4162156367')
('리얼 몽골 [2022~2023 최신판]', 'http://www.hanbit.co.kr/store/books/look.php?p_code=B2847674054')
('IT CookBook, 원리로 이해하는 전력전자공학(2판)', 'http://www.hanbit.co.kr/store/books/look.php?p_code=B5156679966')
('트러스트', 'http://www.hanbit.co.kr/store/books/look.php?p_code=B6189263297')
('리얼 싱가포르 [2022~2023 최신판]', 'http://www.hanbit.co.kr/store/books/look.php?p_code=B8806843023')
('구글 앱스 스크립트 완벽 가이드', 'http://www.hanbit.co.kr/store/books

---
# scrape_by_lxml

In [1]:
!pip install lxml
!pip install cssselect

In [3]:
import lxml.html

In [16]:
# HTML 파일을 읽어 들이고, getroot() 메서드로 HtmlElement 객체 생성
tree = lxml.html.parse('full_book_list.html')
html = tree.getroot()

In [25]:
# cssselect() 메서드로 a 요소의 리스트를 추출 및 반복 수행
count = 0
for a in html.cssselect('a'):
    # href 속성과 글자를 추출합니다.
    print(a.get('href'), a.text)
    count += 1
    if count > 80:
        break
    

https://www.hanbit.co.kr/store/books/full_book_list.html#gnb None
https://www.hanbit.co.kr/store/books/full_book_list.html#top_search None
https://www.hanbit.co.kr/store/books/full_book_list.html#container None
https://www.hanbit.co.kr/index.html None
https://www.hanbit.co.kr/media/ 한빛미디어
https://www.hanbit.co.kr/academy/ 한빛아카데미
https://www.hanbit.co.kr/biz/ 한빛비즈
https://www.hanbit.co.kr/life/ 한빛라이프
https://www.hanbit.co.kr/edu/ 한빛에듀
https://www.hanbit.co.kr/realtime/ 리얼타임
https://www.hanbit.co.kr/textbook/ 한빛정보교과서
https://www.hanbit.co.kr/rent/ 한빛대관서비스
https://www.hanbit.co.kr/member/login.html 로그인
https://www.hanbit.co.kr/member/member_agree.html 회원가입
https://www.hanbit.co.kr/myhanbit/myhanbit.html 마이한빛
https://www.hanbit.co.kr/myhanbit/cart.html 장바구니
https://www.hanbit.co.kr/publisher/foreignrights.html?lang=e ENGLISH
https://www.hanbit.co.kr/index.html 한빛출판네트워크
https://www.hanbit.co.kr/brand/brand_submain.html BRAND
https://www.hanbit.co.kr/channel/channel_submain.html Channel.H
ht

---
# scrape_by_bs4

In [18]:
!pip install beautifulsoup4

In [19]:
from bs4 import BeautifulSoup

In [21]:
# html 파일 읽어 들이고 BeautifulSoup 객체를 생성
with open('full_book_list.html', encoding='utf-8') as f:
    soup = BeautifulSoup(f, 'html.parser')

In [27]:
# find_all() 메서드로 a 요소를 추출 및 반복 수행
count = 0
for a in soup.find_all('a'):
    # href 속성과 글자를 추출합니다.
    print(a.get('href'), a.text)
    count += 1
    if count > 80:
        break

https://www.hanbit.co.kr/store/books/full_book_list.html#gnb 메뉴 바로가기
https://www.hanbit.co.kr/store/books/full_book_list.html#top_search 검색 및 카테고리 바로가기
https://www.hanbit.co.kr/store/books/full_book_list.html#container 본문 바로가기
https://www.hanbit.co.kr/index.html HOME
https://www.hanbit.co.kr/media/ 한빛미디어
https://www.hanbit.co.kr/academy/ 한빛아카데미
https://www.hanbit.co.kr/biz/ 한빛비즈
https://www.hanbit.co.kr/life/ 한빛라이프
https://www.hanbit.co.kr/edu/ 한빛에듀
https://www.hanbit.co.kr/realtime/ 리얼타임
https://www.hanbit.co.kr/textbook/ 한빛정보교과서
https://www.hanbit.co.kr/rent/ 한빛대관서비스
https://www.hanbit.co.kr/member/login.html 로그인
https://www.hanbit.co.kr/member/member_agree.html 회원가입
https://www.hanbit.co.kr/myhanbit/myhanbit.html 마이한빛
https://www.hanbit.co.kr/myhanbit/cart.html 장바구니
https://www.hanbit.co.kr/publisher/foreignrights.html?lang=e ENGLISH
https://www.hanbit.co.kr/index.html 한빛출판네트워크
https://www.hanbit.co.kr/brand/brand_submain.html BRAND
https://www.hanbit.co.kr/channel/channel_submain.h